<a href="https://colab.research.google.com/github/tariqulq-cyber/Pemrosesan-teks-praktek/blob/main/preprocesing/pemrosesan_teks_teori_preprocesing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [37]:

import os
import sys

# =======================================================
# 1. KONFIGURASI AWAL
# =======================================================

NAMA_FILE_INPUT = '/content/drive/MyDrive/pemrosesan teks teori/casefolding_growtopia.csv' # Ganti dengan nama file CSV input Anda
NAMA_FILE_OUTPUT = 'preprocesing_growtopia.csv' # Nama file CSV output
KOLOM_TEKS = 'content' # Nama kolom yang berisi teks untuk diproses

# --- KAMUS NORMALISASI KATA (Contoh) ---
# Tambahkan lebih banyak pasangan kata sesuai kebutuhan Anda.
normalisasi_map = {
    'gaje': 'tidak jelas',
    'bgt': 'banget',
    'bikin': 'membuat',
    'ga': 'tidak',
    'udh': 'sudah',
    'mabar': 'main bareng',
    'krn': 'karena',
    'cpt': 'cepat',
    'kzl': 'kesal',
    'tp': 'tapi'
}

# --- Fungsi Normalisasi Kata ---
def normalisasi_kata(teks, mapping):
    """Mengganti kata tidak baku dalam teks berdasarkan mapping dictionary."""
    if pd.isna(teks):
        return "" # Mengembalikan string kosong jika nilai hilang (NaN)

    list_kata = teks.split()
    # Mengganti kata tidak baku
    kata_normal = [mapping.get(kata, kata) for kata in list_kata]

    return " ".join(kata_normal)

print("=======================================================")
print(f"Memulai Preprocessing untuk file: {NAMA_FILE_INPUT}")
print("=======================================================")


# =======================================================
# 2. MUAT DATA DAN PENANGANAN AWAL
# =======================================================
try:
    # Membaca file CSV
    df = pd.read_csv(NAMA_FILE_INPUT)
    print(f"[INFO] Data berhasil dimuat. Total baris: {len(df)}")

    # Cek apakah kolom teks yang dimaksud ada
    if KOLOM_TEKS not in df.columns:
        print(f"[ERROR] Kolom '{KOLOM_TEKS}' tidak ditemukan. Mohon periksa nama kolom.")
        sys.exit(1)

    # Isi nilai yang hilang (NaN) di kolom teks dengan string kosong sebelum proses
    df[KOLOM_TEKS] = df[KOLOM_TEKS].fillna('')

except FileNotFoundError:
    print(f"[ERROR] File '{NAMA_FILE_INPUT}' tidak ditemukan. Pastikan file ada di folder ini.")
    sys.exit(1)
except Exception as e:
    print(f"[ERROR] Gagal memuat file CSV: {e}")
    sys.exit(1)


# =======================================================
# 3. HAPUS DUPLIKAT (DUPLICATE REMOVAL)
# =======================================================

total_awal = len(df)

# Hapus duplikat berdasarkan kolom teks saja (untuk mempertahankan data user/waktu yang berbeda)
df.drop_duplicates(subset=[KOLOM_TEKS], inplace=True)

jumlah_terhapus = total_awal - len(df)
print(f"[INFO] Hapus Duplikat selesai. Terhapus: {jumlah_terhapus} baris.")
print(f"[INFO] Total baris setelah hapus duplikat: {len(df)}")


# =======================================================
# 4. NORMALISASI KATA (WORD NORMALIZATION)
# =======================================================

# 4.1. Casefolding (Jika kolom input belum di-casefold, lakukan sekarang)
# Karena diasumsikan input sudah di-casefold, ini hanya untuk memastikan.
df[KOLOM_TEKS] = df[KOLOM_TEKS].astype(str).str.lower()
print(f"[INFO] Kolom '{KOLOM_TEKS}' dipastikan dalam huruf kecil (casefolded).")

# 4.2. Penerapan Normalisasi Kata
# Buat kolom baru bernama 'content_processed'
df['content_processed'] = df[KOLOM_TEKS].apply(
    lambda x: normalisasi_kata(x, normalisasi_map)
)
print("[INFO] Normalisasi kata selesai. Hasil disimpan di kolom 'content_processed'.")


# =======================================================
# 5. EKSPOR DATA KE CSV BARU
# =======================================================

# Kolom yang akan diekspor: semua kolom asli + kolom yang sudah diproses
kolom_output = list(df.columns)

print("\n[PREVIEW] Perbandingan data yang diproses (5 Baris Teratas):")
print(df[[KOLOM_TEKS, 'content_processed']].head().to_string(index=False))

try:
    # Pastikan Anda mengekspor DataFrame dengan kolom yang sudah diproses
    with open(NAMA_FILE_OUTPUT, 'w', newline='', encoding='utf-8') as f:
        df.to_csv(
            f,
            index=False,
            encoding='utf-8'
        )

    print(f"\n[SUKSES EKSPOR] Data Preprocessing selesai dan disimpan ke file:")
    print(f"{os.path.abspath(NAMA_FILE_OUTPUT)}")
    print("=======================================================")

except Exception as e:
    # LOGGING ERROR KRITIS UNTUK DIAGNOSIS
    print("=======================================================")
    print("\n[!!! KEGAGALAN KRITIS EKSPOR CSV !!!]")
    print(f"Tipe Kesalahan: {type(e).__name__}")
    print(f"Detail Kesalahan: {e}")
    print("\nSOLUSI (Wajib Dicoba):")
    print("1. Coba ganti NAMA_FILE_OUTPUT dengan jalur ABSOLUT (e.g., C:\\Users\\...\\nama.csv).")
    print("2. Pastikan file output TIDAK sedang dibuka di program lain (Excel, editor teks).")
    print("3. Coba jalankan terminal/CMD Anda sebagai ADMINISTRATOR.")
    print("=======================================================")

Memulai Preprocessing untuk file: /content/drive/MyDrive/pemrosesan teks teori/casefolding_growtopia.csv
[INFO] Data berhasil dimuat. Total baris: 1000
[INFO] Hapus Duplikat selesai. Terhapus: 50 baris.
[INFO] Total baris setelah hapus duplikat: 950
[INFO] Kolom 'content' dipastikan dalam huruf kecil (casefolded).
[INFO] Normalisasi kata selesai. Hasil disimpan di kolom 'content_processed'.

[PREVIEW] Perbandingan data yang diproses (5 Baris Teratas):
                                                                                                                                                   content                                                                                                                                          content_processed
                                                                              game nya susah banget anj cuman buat login bikin akun di persulit game konfl                                                                             gam